In [1]:
import torch
from transformers import BertTokenizer
from model import BertForMultiLabelClassification
tokenizer = BertTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-original")
model = BertForMultiLabelClassification.from_pretrained("monologg/bert-base-cased-goemotions-original")

/home/kwakaba/.pyenv/versions/3.8.16/envs/hitl-xai/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model.bert.encoder.layer[0]

BertLayer(
  (attention): BertAttention(
    (self): BertSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [3]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output
    return hook

model.bert.encoder.layer[0].register_forward_hook(get_activation('enc_layer0'))

In [4]:
x = tokenizer('I love this!')

In [5]:
text = 'I love this!'
x = tokenizer(text)
output = model(torch.tensor([x['input_ids']]))

In [6]:
print(activation)

{'enc_layer0': (tensor([[[ 0.1944,  0.0351, -0.0609,  ..., -0.0459,  0.0716, -0.0028],
         [-1.0277, -0.3622,  0.1901,  ...,  1.1369, -0.9902,  0.6291],
         [ 0.5544,  0.6922, -0.7243,  ..., -1.0739, -0.8291,  0.1846],
         [-1.1916, -0.9952, -1.1789,  ...,  0.1218,  0.5779,  0.0331],
         [-0.8757,  0.7886,  0.3936,  ...,  0.3971, -0.8927, -0.1645],
         [-0.1624,  0.1472, -0.1026,  ...,  0.3953, -0.7019,  0.0187]]],
       grad_fn=<NativeLayerNormBackward0>),)}
